# Practica 7: Modelos del lenguaje

## Objetivos

- Crear modelos del lenguaje a partir de un corpus en inglés
    - Modelo de bigramas
    - Modelo de trigramas

> Un modelo del lenguaje es un modelo estadístico que asigna probabilidades a cadenas dentro de un lenguaje - Jurafsky, 2000

$$ \mu = (\Sigma, A, \Pi)$$

Donde:
- $\mu$ es el modelo del lenguaje
- $\Sigma$ es el vocabulario
- $A$ es el tensor que guarda las probabilidades de trancisiones
- $\Pi$ guarda las probabilidades iniciales

- Este modelo busca estimar la probabilidad de una secuencia de tokens
- Pueden ser palabras, caracteres o tokens
- Se pueden considerar varios escenarios para la creación de estos modelos

## Aplicaciones

- Traducción automática
- Completado de texto
- Generación de texto

## De los bigramas a los n-gramas

- Para bigramas tenemos la propiedad de Markov
- Para $n > 2$ las palabras dependen de mas elementos
    - Trigramas
    - 4-gramas
- En general para un modelo de n-gramas se toman en cuenta $n-1$ elementos

## Obteniendo y preprocesando el texto

Vamos a trabajar con el corpus gutenberg disponible en el paquete NLTK

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

In [ ]:
from nltk.corpus import gutenberg

gutenberg.fileids()

In [ ]:
gutenberg.sents(fileids="bible-kjv.txt")[:3]

El preprocesamiento consistira en eliminar signos de puntuación y dejar todas las palabras en minúsculas
- Cabe señalar que, dependiendo de la aplicación puede que sea necesario mantener los signos de puntuación como elementos del vocabulario
- Para simplificar en la práctica no se considerarán

In [ ]:
import re

def preprocess_corpus(corpus: list[list[str]]) -> list:
    clean_corpus = []
    for sent in corpus:
        clean_corpus.append([word.lower() for word in sent if re.match("^(?![0-9]+$)[\w\s]+$", word)])
    return clean_corpus

corpus = preprocess_corpus(gutenberg.sents(fileids="bible-kjv.txt"))

In [ ]:
len(corpus)

In [ ]:
corpus = corpus[:500]

In [ ]:
corpus[-10:]

Vamos a partir el corpus en dos secciones. Una para train con la que entrenaremos el modelo y otra para probar el modelo

In [ ]:
from sklearn.model_selection import train_test_split

corpus_train, corpus_test = train_test_split(corpus, test_size=0.3)

len(corpus_train) + len(corpus_test) == len(corpus)

In [ ]:
print("Train len:", len(corpus_train), "test len:", len(corpus_test))

Nuestro modelo del lenguaje requiere que pasemos nuestras palabras a indices numericos. Utilizaremos enteros para estimar el modelo.
Crearemos dos diccionarios:
    1. el primero tomara la palabra y lo convertira a indice (Para acceder a las probabilidades del modelo)
    2. El segundo tomará los indices y los convertira de vuelta a palabras (Nos ayudará a recuperar las palabras a partir de los índices del modelo)

In [ ]:
from collections import defaultdict, Counter

def vocabulary_factory():
    """Function that create a vocabulary

    Default method when a key is not in the dictionary changed to be the
    current lenght of the dictionary to provide a unique index for each
    new key.

    Example:
    >> vocab['test']
    0
    >> vocab['other']
    1
    >> vocab['test']
    0
    """
    vocab = defaultdict()
    vocab.default_factory = lambda: len(vocab)
    return vocab

def word_to_index(corpus: list[list[str]], vocab: defaultdict) -> list[int]:
    """Function that maps each word in a corpus to a unique index"""
    for sent in corpus:
        yield [vocab[word] for word in sent]


In [ ]:
vocab = vocabulary_factory()

In [ ]:
indexed_sents = list(word_to_index(corpus_train, vocab))

In [ ]:
indexed_sents[:4]

El vocabulario aun no esta completo. Debemos agregar etiquetas que indiquen BOS (Beginning Of String) y EOS (End Of String). Debemos añadirlos a cada oración en nuestro corpus de entrenamiento:

$$<s> w_1 ... w_k </s>$$

De esta forma, podremos obtener probabilidades inciales y transiciones terminales (aquellas que van hacía el símbolo de termino EOS).

Estas etiquetas con arbitrarias, usaremos entonces $BOS = <s>$ y $EOS = </s>$

In [ ]:
BOS = "<s>"
EOS = "</s>"

BOS_IDX = max(vocab.values())+2
EOS_IDX = max(vocab.values())+1

vocab[BOS] = BOS_IDX
vocab[EOS] = EOS_IDX

indexed_corpus_train = [[BOS_IDX] + sent + [EOS_IDX] for sent in indexed_sents]

In [ ]:
def get_index_to_word(vocab: defaultdict) -> dict:
    """Map indices as keys and words as values from a vocabulary"""
    return {index: word for word, index in vocab.items()}

In [ ]:
vocab_words = get_index_to_word(vocab)

In [ ]:
vocab_words[100]

In [ ]:
len(vocab)

## Estimación del modelo de n-gramas

Una vez preprocesadas las cadenas pasaremos a estimar el modelo. Para esta estimación, tomaremos en cuenta dos parámetros:

*   El tamaño de n-gramas; es decir, qué tantos elementos previos consideraremos para estimar la probabilidad de que ocurra una palabra.
    - bigramas
    - trigramas
    - etc
*   El elemento $\lambda$ para estimar la probabilidad con smoothing de Lidstone. En ese sentido, dado un n-grama $w_{i-n+1} ... w_{i-1} w_i$ estimaremos la probabilidad como:

$$p(w_i|w_{i-1}...w_{i-n+1}) = \frac{C(w_{i-n+1} ... w_{i-1} w_i) + \lambda}{C(w_{i-n+1} ... w_{i-1}) + \lambda N}$$

donde $N$ es el tamaño del vocabulario.

In [ ]:
import numpy as np
from itertools import chain

def get_n_grams(indexed_sents: list[list[str]], n=2) -> chain:
    return chain(*[zip(*[sent[i:] for i in range(n)]) for sent in indexed_sents])

def get_model(sents: list[list[str]], vocabulary: defaultdict, n: int=2, l: float=1.0) -> tuple:

    # Get n_grams
    n_grams = get_n_grams(sents, n)

    # Get n_grams frequencies
    freq_n_grams = Counter(n_grams)

    # Get vocabulary length (without BOS/EOS)
    N = len(vocabulary) - 2
    # Calculate tensor dimentions for transition probabilities
    # For columns (conditional word) we consider the EOS element so we add 1
    dim = (N,)*(n-1) + (N+1,)

    # Transition tensor
    A = np.zeros(dim)
    # Initial Probabilities
    Pi = np.zeros(N)

    for n_gram, frec in freq_n_grams.items():
      # Fill the tensor with frequencies
      if n_gram[0] != BOS_IDX:
          A[n_gram] = frec
      # Getting initial frequencies
      elif n_gram[0] == BOS_IDX and n_gram[1] != EOS_IDX:
          Pi[n_gram[1]] = frec

    # Calculating probabilities from frequencies
    # We consider the parameter `l` for Lidstone Smoothing
    for h, b in enumerate(A):
      A[h] = ((b+l).T/(b+l).sum(n-2)).T

    # Calculating initial probabilities
    Pi = (Pi+l)/(Pi+l).sum(0)

    # We get our model
    return A, Pi

### Detalles de implementación

In [ ]:
bigrams = get_n_grams(indexed_corpus_train, n=2)

In [ ]:
list(bigrams)[:3]

In [ ]:
for i, b in enumerate(bigrams):
    print(b)
    print(vocab_words[b[0]], vocab_words[b[1]])
    if i == 10:
        break

In [ ]:
N = len(vocab) - 2
n = 3
dim = (N,)*(n-1) + (N+1,)

In [ ]:
(N,)*(n-1) + (N+1,)

## Estimación del modelo

Estimaremos un modelo de trigramas con $λ = 1$, es decir con smoothing Laplaciano

In [ ]:
%%time
trigram_model = get_model(indexed_corpus_train, vocab, n=3, l=1)

In [ ]:
A_trigram = trigram_model[0]
print("Tensor dimention", A_trigram.shape)
print("Suma de probabilidades")
print(A_trigram.sum(1))

Estimando un modelo de bigramas con $λ = 1$

In [ ]:
%%time
bigram_model = get_model(indexed_corpus_train, vocab, n=2, l=1)

In [ ]:
A_bigram = bigram_model[0]
print("Tensor dimention", A_bigram.shape)
print("Suma de probabilidades")
print(A_bigram.sum(1))

## Aplicaciones

1. Obtener la probabilidad de una cadena
2. Predecir una palabra siguiente
3. Generación de texto

Para determinar la probabilidad, utilizaremos la función:

$$p(w_1 ... w_k) = \prod_{i=1}^k p(w_i|w_{i-1} ... w_{i-n+1})$$

Dado que las cadenas pueden extenderse y las probabilidades son pequeñas, es posible que la probabilidad se haga tan pequeña que aparezca como un cero. Para evitar esto, utilizaremos probabilidad logarítimicada, dada por:

$$\log p(w_1 ... w_k) = \sum_{i=1}^k \log p(w_i|w_{i-1} ... w_{i-n+1})$$

### 1. Obtener la probabilidad de una cadena

In [ ]:
def get_sent_probability(sentence: str, vocab: defaultdict, model: tuple) -> float:
    A, Pi = model
    # Getting the n from n-grams
    n = len(A.shape)
    indexed_sentence = [vocab[word] for word in sentence.split()]
    first_indexed_word = indexed_sentence[0]
    # Getting initial probability
    try:
        probability = np.log(Pi[first_indexed_word])
    except:
        print(f"[WARN] OOV for word as BOS with index={first_indexed_word}")
        probability = 0.0

    # Getting n-grams of the sentence
    n_grams = get_n_grams([indexed_sentence], n)
    for n_gram in n_grams:
        try:
          probability += np.log(A[n_gram])
        except:
          print(f"[WARN] OOV for n_gram={n_gram}")
          probability += 0.0

    return probability

In [ ]:
sentence = " ".join(corpus_train[-1])
print(f"La probabilidad de la cadena: <{sentence}>")
print(f"\t\t Modelo de trigramas: ", np.exp(get_sent_probability(sentence, vocab, trigram_model)))
print(f"\t\t Modelo de bigramas: ", np.exp(get_sent_probability(sentence, vocab, bigram_model)))

In [ ]:
TEST_SENTENCE = "and god said"

In [ ]:
np.exp(get_sent_probability(TEST_SENTENCE, vocab, bigram_model))

### 2. Predecir la palabra siguiente

In [ ]:
def predict_next_word(sentence: str, vocab: defaultdict, vocab_words: dict, model: tuple) -> str:
    A, Pi = model
    history = len(A.shape) - 1
    indexed_sentence = [vocab[word] for word in sentence.split()]
    prev_n_gram = tuple(indexed_sentence[-history:])
    probability = get_sent_probability(sentence, vocab, model)
    next_word = np.argmax(probability + np.log(A[prev_n_gram]))
    return vocab_words[next_word]

In [ ]:
predict_next_word(TEST_SENTENCE, vocab, vocab_words, trigram_model)

In [ ]:
predict_next_word(TEST_SENTENCE, vocab, vocab_words, bigram_model)

### 3. Generación de texto

Iterando sobre la función anterior podemos producir texto. Nustro algoritmo buscara el token *EOS* para detenerse o despues de producir *N* tokens.

In [ ]:
def generate_laguage(sentence: str, vocab: defaultdict, vocab_words: dict, model: tuple, limit: int) -> str:
    next_word = ""
    result = sentence
    i = 0
    while next_word != "</s>":
        next_word = predict_next_word(result, vocab, vocab_words, model)
        result += " " + next_word
        i += 1
        if i == limit:
            break

    return result

In [ ]:
print(f"Modelo de trigramas: {TEST_SENTENCE}")
generate_laguage(TEST_SENTENCE, vocab, vocab_words, trigram_model, 100)

In [ ]:
print("Modelo de bigramas")
generate_laguage("and god", vocab, vocab_words, bigram_model, 10)

## Práctica 7: Evaluación de modelos de lenguaje

**Fecha de entrega**: 5 de noviembre de 2023

La calidad de un modelo del lenguaje puede ser evaluado por medio de su perplejidad

- Investigar como calcular la perplejidad de un modelo del lenguaje y como evaluarlo con esa medida
    - Incluir en el `README.md` una sintesis de esta investigación (Un par de parrafos)
- Crear un par de modelos del lenguaje usando un **corpus en español**
    - Corpus: El Quijote
        - URL: https://www.gutenberg.org/ebooks/2000
    - Modelo de n-gramas con `n = [2, 3]`
    - Hold out con `test = 30%` y `train = 70%`
- Evaluar los modelos y reportar la perplejidad de cada modelo
  - Comparar los resultados entre los diferentes modelos del lenguaje (bigramas, trigramas)
  - ¿Cual fue el modelo mejor evaluado? ¿Porqué?
